# Visualize the molecule

In [215]:
from rdkit import Chem
import py3Dmol
mol_folder = './Molecules/'
mol_fname = '20A12Rot_1.mol2'
mol_filepath = mol_folder + mol_fname
mol = Chem.MolFromMol2File(mol_filepath)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Input Variables

In [216]:
n_atoms_mol = mol.GetNumAtoms()  # no. of atoms in the molecule (excluding hydrogen atoms)
print('No. of atoms: ', n_atoms_mol)
conformers = mol.GetConformers()
conf = conformers[0]
coords_mol = {}  # Coordinates of the atoms
for i in range(n_atoms_mol):
    coords_mol[i] = list(conf.GetAtomPosition(i))
print('Coordinates of atoms:')
for i in coords_mol:
    print(i, '\t:', coords_mol[i])

No. of atoms:  20
Coordinates of atoms:
0 	: [1.784, 2.211, -8.17]
1 	: [0.729, 1.499, -7.537]
2 	: [1.214, 0.656, -6.496]
3 	: [0.028, -0.063, -5.869]
4 	: [0.479, -0.967, -4.735]
5 	: [-0.898, -1.847, -4.01]
6 	: [-1.863, -0.872, -3.547]
7 	: [-1.28, -2.926, -4.895]
8 	: [-0.154, -2.535, -2.651]
9 	: [0.65, -3.751, -2.89]
10 	: [-0.992, -2.638, -1.429]
11 	: [-0.501, -1.77, -0.309]
12 	: [0.914, -1.515, -0.268]
13 	: [0.067, -0.388, -0.533]
14 	: [0.007, 0.673, 0.525]
15 	: [0.549, 0.264, 1.896]
16 	: [0.437, 1.41, 2.904]
17 	: [0.997, 1.004, 4.268]
18 	: [0.871, 2.119, 5.266]
19 	: [0.159, 2.036, 6.397]


In [217]:
input_folder = './Input_Rot_2/'
input_fname = mol_fname[:-5] + '_input.txt'
input_filepath = input_folder + input_fname
input_lines = open(input_filepath, 'r').readlines()
torsional_bonds_mol = eval(input_lines[0])
median_list_mol = []
for atoms in input_lines[1].split(','):
    median_list_mol.append(int(atoms))
coords_rotation_mol = eval(input_lines[2])
distance_pairs_mol = eval(input_lines[3])
print(torsional_bonds_mol)
print(median_list_mol)
print(coords_rotation_mol)
print(distance_pairs_mol)

{0: (4, 5), 1: (10, 11)}
[2, 4, 5, 7, 9, 10, 11, 15]
{6: [0], 7: [0], 8: [0], 9: [0], 10: [0], 11: [0], 12: [0, 1], 13: [0, 1], 14: [0, 1], 15: [0, 1], 16: [0, 1], 17: [0, 1], 18: [0, 1], 19: [0, 1]}
{2: [7, 9, 10, 11, 15], 4: [7, 9, 10, 11, 15], 5: [7, 9, 10, 11, 15], 7: [15], 9: [15], 10: [15], 11: [15], 15: []}


In [218]:
# torsional_bonds_mol = { 0: (9, 10), 1: (19, 20)  }  # Torsional bond numbers and their respective coordinate ends
# median_list_mol = [2, 6, 9, 10, 14, 19, 20, 23]  # List of atoms (median) considered for the HUBO construction
# coords_rotation_mol = {11:[0], 12:[0], 13:[0], 14:[0], 15:[0], 16:[0], 17:[0], 18:[0], 19:[0], 20:[0], 21:[0,1], 22:[0,1], 23:[0,1], 24:[0,1] }  # atom index and the rotatable bond numbers list that affect them
# distance_pairs_mol = {2:[14, 19, 20, 23], 6:[14, 19, 20, 23], 9:[14, 19, 20, 23], 10:[23], 14:[23], 19:[23], 20:[], 23:[] }  # indices dictionary to find distances

In [219]:
import copy

coords_median = {}
for i, index in enumerate(median_list_mol):
    coords_median[i] = coords_mol[index]
coords_median_2 = copy.deepcopy(coords_median)
print('coords_median: ', coords_median)

torsional_bonds = {}
for i in torsional_bonds_mol:
    torsional_bonds[i] = (median_list_mol.index(torsional_bonds_mol[i][0]), median_list_mol.index(torsional_bonds_mol[i][1]))
print('\ntorsional_bonds: ', torsional_bonds)

coords_rotation_dict = {}
for atom in median_list_mol:
    if atom in coords_rotation_mol.keys():
        coords_rotation_dict[median_list_mol.index(atom)] = coords_rotation_mol[atom]
print('\ncoords_rotation_dict: ', coords_rotation_dict)

distance_pairs = []
for x, y in distance_pairs_mol.items():
    u = median_list_mol.index(x)
    for i in y:
        v = median_list_mol.index(i)
        distance_pairs.append((u, v))
print('\ndistance pairs: ', distance_pairs)



coords_median:  {0: [1.214, 0.656, -6.496], 1: [0.479, -0.967, -4.735], 2: [-0.898, -1.847, -4.01], 3: [-1.28, -2.926, -4.895], 4: [0.65, -3.751, -2.89], 5: [-0.992, -2.638, -1.429], 6: [-0.501, -1.77, -0.309], 7: [0.549, 0.264, 1.896]}

torsional_bonds:  {0: (1, 2), 1: (5, 6)}

coords_rotation_dict:  {3: [0], 4: [0], 5: [0], 6: [0], 7: [0, 1]}

distance pairs:  [(0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7)]


In [220]:
import sympy as sp
n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles
x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)


# Function to generate hard constraint

In [221]:
# generate hard constraint
def generate_hard_hubo():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

# Functions to rotate coordinates with hubo variables

In [222]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [223]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [224]:
def rotation_matrix_hubo(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [225]:
from sympy.matrices import ones, eye
import time
start = time.time()

for i in coords_rotation_dict:
    rot_mat = eye(4, 4)
    for bond_no in coords_rotation_dict[i]:
        temp_rot_mat = rotation_matrix_hubo(coords_median[torsional_bonds[bond_no][0]], coords_median[torsional_bonds[bond_no][1]], bond_no)
        rot_mat = rot_mat * temp_rot_mat
    coords_median_2[i] = rotate_coordinates(rot_mat, coords_median[i])

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  4.566805362701416 seconds


# Functions to generate the optimization contraint in hubo

In [226]:
def distance_squared(first_coords, second_coords):
    dis_sq = (second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 + (second_coords[2] - first_coords[2])**2
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_median_2[pair[0]], coords_median_2[pair[1]])
    return distance_sq.expand()

In [227]:
start = time.time()

hard_constraint = generate_hard_hubo()
distance_constraint = generate_distance_hubo()
hubo_expr = sp.expand(hard_constraint - distance_constraint)

end = time.time()
print("The time of execution: ", (end-start), "seconds")

The time of execution:  84.97041845321655 seconds


# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [228]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [229]:
import re
hubo_dict = hubo_expr_to_dict()

In [230]:
hubo_folder = './Hubo_Rot_2/'
hubo_fname = mol_fname[:-5] + '_hubo.txt'
hubo_filepath = hubo_folder + hubo_fname
f = open(hubo_filepath, "w")
print('full hubo dictionary written in file - ' + hubo_filepath)
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - ./Hubo_Rot_2/20A12Rot_1_hubo.txt


In [231]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(hubo_filepath, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(hubo_filepath, 'r').read()
HUBO=eval(read_dictionary)

In [232]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 1369


In [233]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [234]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [235]:
import dimod
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [236]:
def find_bond_theta_soln(solution):
    thetas = generate_thetas()
    bond_theta_soln = {}
    for key, value in solution.items():
        if solution[key] == 1:
            bond_theta_soln[key//n_angles] = thetas[key%n_angles]
    return bond_theta_soln

In [237]:
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_size=10
start = time.time()
sampleset = sampler.sample(bqm, num_reads=sample_size)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset)

The time of execution:  0.01174783706665039 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15      energy num_oc.
8  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0 -530.164013       1
6  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0 -527.063561       1
5  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -523.744433       1
4  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0 -482.641003       1
0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1 -482.118454       1
3  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  -478.17086       1
7  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0  0 -465.261626       1
9  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0 -465.181019       1
2  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0 -462.795309       1
1  0  0  1  1  0  0  0  0  0  0  0  0  0  0  1  0 5478.993564       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [238]:
sa_soln = sampleset.first.sample
print(sa_soln)
bond_theta_soln = find_bond_theta_soln(sa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0}
{0: 7*pi/4, 1: 3*pi/4}


In [239]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
start = time.time()
sampleset = sampler.sample(bqm, num_reads=1000)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset.slice(10))

The time of execution:  0.23360204696655273 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15      energy num_oc. ...
0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0 -530.164013       9 ...
1  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0 -529.219029       3 ...
2  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -529.138422      10 ...
3  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0 -527.155735       4 ...
4  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0 -527.063561       4 ...
5  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -526.752712      13 ...
6  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0 -525.807727      11 ...
7  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -525.727121       9 ...
8  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0 -525.503309       5 ...
9  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0 -525.453562       7 ...
['BINARY', 10 rows, 75 samples, 16 variables]


In [240]:
qa_soln = sampleset.first.sample
print(qa_soln)
bond_theta_soln = find_bond_theta_soln(qa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0}
{0: 7*pi/4, 1: 3*pi/4}


In [241]:
def rotation_matrix_new_coords(first_coords, second_coords, bond_no, soln_theta):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    c_theta = sp.cos(soln_theta)
    s_theta = sp.sin(soln_theta)
    index = n_angles * bond_no
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = ((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[0, 1] = ((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 2] = ((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 3] = (((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 0] = ((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 1] = ((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[1, 2] = ((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 3] = (((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 0] = ((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 1] = ((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 2] = ((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[2, 3] = (((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq).evalf()
    return rotation_matrix

In [242]:
def compute_new_coords(solution):
    final_coords = copy.deepcopy(coords_mol)
    for i in coords_rotation_mol:
        if len(coords_rotation_mol[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_mol[i]:
                temp_rot_mat = rotation_matrix_new_coords(coords_mol[torsional_bonds_mol[bond_no][0]], coords_mol[torsional_bonds_mol[bond_no][1]], bond_no, solution[bond_no])
                rot_mat = rot_mat * temp_rot_mat
            final_coords[i] = rotate_coordinates(rot_mat, coords_mol[i])
    return final_coords


In [243]:
compute_new_coords(bond_theta_soln)

{0: [1.784, 2.211, -8.17],
 1: [0.729, 1.499, -7.537],
 2: [1.214, 0.656, -6.496],
 3: [0.028, -0.063, -5.869],
 4: [0.479, -0.967, -4.735],
 5: [-0.898, -1.847, -4.01],
 6: [-1.24138894337648, -1.19805583668886, -2.76213201560775],
 7: [-1.89078372368214, -2.08567535289794, -5.03508758353171],
 8: [-0.167636577116067, -3.33265446848125, -3.64508758476183],
 9: [-0.00446405072603651, -4.27591935725347, -4.77017383756249],
 10: [-0.633242671029962, -4.00254032910638, -2.40387813465086],
 11: [0.431914904945402, -4.08891906361392, -1.35179303706266],
 12: [0.305431979216954, -3.19058104834707, -0.235584329057931],
 13: [0.161338928517442, -4.58938630731519, 0.0478659138829259],
 14: [1.23436594209187, -5.22136665551148, 0.883269329983943],
 15: [2.52366914678451, -4.40909568368011, 1.01964790120906],
 16: [3.55460284055266, -5.14069743478224, 1.88212027574512],
 17: [4.83482783433540, -4.31759884913246, 2.03200250816544],
 18: [5.85707006347297, -5.04043553014971, 2.86123887829389],
 19:

In [244]:
def find_volume_change(old_coords, new_coords):
    n = len(old_coords)
    old_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            old_distance_sq += distance_squared(old_coords[i], old_coords[j])
    
    new_distance_sq = 0
    for i in range(n-1):
        for j in range(i+1, n):
            new_distance_sq += distance_squared(new_coords[i], new_coords[j])
    # print("new volume: ", new_distance_sq.evalf())
    # print("old volume: ", old_distance_sq)
    # print('change in volume: ', new_distance_sq.evalf() - old_distance_sq)
    return new_distance_sq.evalf() - old_distance_sq

In [245]:
# bond_theta_soln = {0: 0, 1: sp.pi}
# bond_theta_soln = {0: sp.pi/4}
# bond_theta_soln = {0: 2*sp.pi/4}
# bond_theta_soln = {0: 3*sp.pi/4}
# bond_theta_soln = {0: 4*sp.pi/4}
# bond_theta_soln = {0: 5*sp.pi/4}
# bond_theta_soln = {0: 6*sp.pi/4}
# bond_theta_soln = {0: 7*sp.pi/4}

In [246]:
new_coords = compute_new_coords(bond_theta_soln)
volume_change = find_volume_change(coords_mol, new_coords)
print('change in volume: ', volume_change)

change in volume:  575.708976274458


In [247]:
# brute force solution
start = time.time()
thetas = generate_thetas()
best_volume_change = 0
best_soln = {}
for i in range(8):
    for j in range(8):
        bond_theta_soln = {0: thetas[i], 1: thetas[j]}
        new_coords = compute_new_coords(bond_theta_soln)
        vol_difference = find_volume_change(coords_mol, new_coords)
        if vol_difference > best_volume_change:
            best_volume_change = vol_difference
            best_soln = bond_theta_soln
end = time.time()
print("The time of execution: ", (end-start), "s")    
print('best solution: ', best_soln)
print('best volume change: ', best_volume_change)




The time of execution:  16.138725996017456 s
best solution:  {0: 7*pi/4, 1: pi/2}
best volume change:  630.390224485020


In [105]:
from rdkit.Geometry import Point3D
conf = mol.GetConformer()
for i in range(len(coords_mol)):
    x, y, z = new_coords[i]
    conf.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))

In [106]:
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [107]:
mol_file = './Molecules/25A12Rot_2.mol2'
mol = Chem.MolFromMol2File(mol_file)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol